In [1]:
import os
from collections import defaultdict
import random
import sys
from munkres import Munkres
import numpy
import networkx as nx

In [2]:
from networkx.algorithms import community


In [3]:
from networkx.algorithms.community import greedy_modularity_communities

In [6]:
from sklearn.cluster import SpectralClustering

In [15]:
import sklearn

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def read_features_trial_2(featname):
    features = {}
    featnames = []
    nodeIdSim = {}
    file= open(featname)
    file_lst = file.readlines()
    fin_rslt = [[None for i  in range(len(file_lst))] for j in range(len(file_lst))]
    for i in range(len(file_lst)):
        lst = file_lst[i].split(" ")
        nodeId1,fts1 = lst[0],lst[1:]
        fts1 = np.asarray(fts1)
        fts1 = fts1.reshape(1,-1) 
        j = 0
        while(j < len(file_lst)):
            nodeIdRel = ()
            lst2 = file_lst[j].split(' ')            
            nodeId2,fts2 = lst2[0],lst2[1:]
            nodeIdRel=(nodeId1,nodeId2)
            fts2 = np.asarray(fts2)
            fts2 = fts2.reshape(1,-1)
            similarity = cosine_similarity(fts1,fts2)
            nodeIdSim[nodeIdRel] = similarity[0][0]
            fin_rslt[i][j] = similarity[0][0]
            j+=1
    return fin_rslt

In [4]:
def loss1(usersPerCircle, usersPerCircleP):
    psize = max(len(usersPerCircle),len(usersPerCircleP)) # Pad the matrix to be square
    mm = numpy.zeros((psize,psize))
    mm2 = numpy.zeros((psize,psize))
    for i in range(psize):
        for j in range(psize):
            circleP = set() # Match to an empty circle (delete all users)
            circle = set() # Match to an empty circle (add all users)
            if (i < len(usersPerCircleP)):
                circleP = usersPerCircleP[i]
            if (j < len(usersPerCircle)):
                circle = usersPerCircle[j]
            nedits = len(circle.union(circleP)) - len(circle.intersection(circleP)) # Compute the edit distance between the two circles
            mm[i][j] = nedits
            mm2[i][j] = nedits

    if psize == 0:
        return 0 # Edge case in case there are no circles
    else:
        m = Munkres()
        #print mm2 # Print the pairwise cost matrix
        indices = m.compute(mm) # Compute the optimal alignment between predicted and groundtruth circles
        editCost = 0
        for row, column in indices:
            editCost += mm2[row][column]
    return int(editCost)


In [12]:
def read_nodeadjlist(filename):
    G = nx.Graph()
    for line in open(filename):
        lst = line.split(' ')
        el, es = lst[0], lst[1][:-1]
        G.add_edge(int(el),int(es))
    return G

In [10]:
G1 = read_nodeadjlist("twitter/78813.edges")

In [13]:
def naive_spec_cluster(G,ego):
    cl = sklearn.cluster.SpectralClustering(affinity="precomputed")
    user_feat = read_features_trial_2("twitter/"+ego+".feat")
    cl.fit(user_feat)
    pred_circles ={}
    for circle,user in zip(cl.labels_,G.nodes()):  # ordering is the same as G.nodes()
        if circle in pred_circles:
            pred_circles[circle].append(user)
        else:
            pred_circles[circle]=[user]
    return pred_circles

In [16]:
predCircle = naive_spec_cluster(G1, "78813")

C:\Users\Rutul Thakkar\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:237: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [17]:
predCircle

{0: [11336782,
  627363,
  89318473,
  14616067,
  992031,
  13055,
  24454827,
  233693208,
  8168192,
  14079172,
  113963,
  16304817,
  353466012,
  5637652,
  27478849,
  14058661,
  5796972,
  14371227,
  14161531,
  821449,
  13753352,
  3375371,
  18296076,
  121887978,
  174853,
  13837292,
  13170222,
  297623133,
  30495974,
  63508150,
  1976611,
  22328084,
  65502805,
  23279911,
  2419,
  250530254,
  71062740,
  1950981,
  138034438,
  14940487,
  105673792,
  6244092,
  378130010,
  55384770,
  20290970,
  18041656],
 7: [3594701,
  782329,
  2195241,
  15416485,
  15023,
  16993577,
  16624991,
  12307282,
  24741685,
  4044361,
  6515122,
  16930986,
  12101862,
  5746402,
  130584631,
  12199652,
  1375251,
  18852278,
  16265198,
  17220817,
  21437000,
  10461992,
  14091119,
  14154082,
  621713,
  6813682,
  15258591,
  15736732,
  1847381,
  13434092,
  14964767,
  16120265,
  6271152,
  15948437,
  14048987,
  229523,
  15294787,
  31354534,
  4958131,
  13535

In [19]:
pred = []
for key, value in predCircle.items():
    pred.append(set(value))
print(pred)

[{8168192, 14616067, 174853, 1950981, 138034438, 13753352, 3375371, 18296076, 5637652, 22328084, 233693208, 20290970, 14371227, 353466012, 992031, 627363, 1976611, 14058661, 23279911, 24454827, 113963, 13170222, 16304817, 63508150, 18041656, 105673792, 27478849, 55384770, 14079172, 14940487, 89318473, 821449, 11336782, 250530254, 71062740, 65502805, 378130010, 297623133, 30495974, 121887978, 5796972, 13837292, 2419, 14161531, 6244092, 13055}, {17885184, 7434252, 223048206, 13535762, 1375251, 161739794, 14976534, 81231895, 19459100, 14964767, 36412963, 130584631, 26955321, 3388481, 21437000, 4044361, 586, 3361871, 1847381, 156001879, 17800797, 13405, 17093224, 5849202, 16186995, 11375732, 5385852, 18498684, 14344319, 179980928, 793219, 18208247, 5963912, 621713, 229523, 15948437, 16511126, 171592863, 15416485, 31354534, 16583846, 16930986, 15023, 6271152, 16439471, 7924912, 198693040, 309206712, 19658936, 14297283, 7921352, 361943754, 9535182, 7684302, 17220817, 14048987, 5746402, 12199

In [24]:
# pred_list = []
# for circle in predCircle:
#     for val in circle:
#         pred_list.append(int(val))


In [26]:
# set_pred = set(pred_list)

In [20]:
# set_pred

In [21]:
def read_circles(filename):
    final_lst = []
    for line in open(filename):
        lst = line.split('\t')
        el, es = lst[0], lst[1:]
        circle  =set()
        for e in es:
            circle.add(int(e))
        final_lst.append(circle)
    return final_lst

In [22]:
gt_circles =read_circles("twitter/78813.circles")

In [23]:
gt_circles

[{586,
  3839,
  113963,
  174853,
  229523,
  793219,
  813491,
  2384071,
  5676102,
  11336782,
  13535762,
  15948437,
  30313925,
  50393960},
 {3839,
  174853,
  621713,
  849131,
  992031,
  2384071,
  5497452,
  5637652,
  5676102,
  9460662,
  11336782,
  11362622,
  12199652,
  14371227,
  14405111,
  14964767,
  15948437,
  27478849,
  50393960}]

In [24]:
loss1(pred, gt_circles)

236